In [14]:
import numpy as np
from numpy.linalg import inv
p = np.array([1,2,3])
print(p)

[1 2 3]


In [15]:
def updated_mean(mean1, var1, mean2, var2):
    new_mean = (mean1 * var2 + mean2 * var1) / (var1 + var2)
    return new_mean

def updated_var(var1, var2):
    new_var = 1 / ((1 / var1) + (1 / var2))
    return new_var

In [16]:
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

In [17]:
# Initial Values
x = 50
x_dot = 5

# Standard Deviations
x_std = 0.5
x_dot_std = 0.2

# Variance
x_var = x_std ** 2
x_dot_var = x_dot_std ** 2

x_cov_x_dot = x_std * x_dot_std
x_dot_cov_x = x_dot_std * x_std

X = np.array([[x], [x_dot]])
P = np.array([[x_var, x_cov_x_dot],
              [x_dot_cov_x, x_dot_var]])

In [18]:


x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 282, 285, 286, 290])

z = np.c_[x_observations, v_observations]

# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6

def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime


def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return np.diag(np.diag(cov_matrix))


# Initial Estimation Covariance Matrix
P = covariance2d(error_est_x, error_est_v)
A = np.array([[1, t],
              [0, 1]])

# Initial State Matrix
X = np.array([[z[0][0]],
              [v]])
n = len(z[0])

for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, professor
    # set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

    print("Kalman Filter State Matrix:\n", X)

Kalman Filter State Matrix:
 [[4272.5]
 [ 282. ]]
Kalman Filter State Matrix:
 [[4553.85054707]
 [ 284.29069767]]
Kalman Filter State Matrix:
 [[4844.15764332]
 [ 286.22522523]]
Kalman Filter State Matrix:
 [[5127.05898493]
 [ 288.55147059]]
